In [2]:

#%%capture
#!git clone https://github.com/ynop/py-ctc-decode
#!pip3 install transformers datasets torchaudio
#!pip3 install https://github.com/kpu/kenlm/archive/master.zip

In [3]:
#!git clone https://github.com/ynop/py-ctc-decode

In [4]:
#!pip install transformers datasets torchaudio

In [5]:
#!pip install https://github.com/kpu/kenlm/archive/master.zip

In [6]:
#%cd py-ctc-decode

In [7]:
# ##########################################################################################################
# ##########################################################################################################
#
# run the following command in the environment in Putty terminal
# !pip install https://github.com/kpu/kenlm/archive/master.zip
#
# ##########################################################################################################
# ##########################################################################################################

In [1]:
cd /srv/scratch/z5313567/thesis/JupyterLab/Language_model

/srv/scratch/z5313567/thesis/JupyterLab/Language_model


In [2]:
import librosa
import numpy as np

In [3]:
data_train_fp = '/srv/scratch/z5313567/thesis/AusKidTalk_local/spontaneous/AusKidTalk_spontaneous_test_only_transcription_filepath.csv'
data_test_fp = '/srv/scratch/z5313567/thesis/AusKidTalk_local/spontaneous/AusKidTalk_spontaneous_test_only_transcription_filepath.csv'
cache_fp = '/srv/scratch/z5313567/thesis/cache'
#model_fp = '/srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk/finetune_20230708_2'
model_fp = '/srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk_scripted_spontaneous_combined/finetune_20230718'

# 1. Decoding audio data with Wav2Vec2 and a language model

In [4]:
from datasets import load_dataset

dataset = load_dataset('csv', 
                        data_files={'train': data_train_fp,
                                'test': data_test_fp},
                        cache_dir = cache_fp)

/home/z5313567/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 69.24it/s]


Dataset csv downloaded and prepared to /srv/scratch/z5313567/thesis/cache/csv/default-f00e5d99dcfcccbc/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 210.21it/s]


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['filepath', 'transcription_clean'],
        num_rows: 24
    })
    test: Dataset({
        features: ['filepath', 'transcription_clean'],
        num_rows: 24
    })
})

In [6]:
def speech_file_to_array_fn(batch):
    #speech_array, sampling_rate = sf.read(batch["filepath"])
    speech_array, sampling_rate = librosa.load(batch['filepath'], sr=16000)
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["transcription_clean"]
    return batch

data = dataset.map(speech_file_to_array_fn, remove_columns=dataset.column_names["train"], num_proc=4)

In [7]:
data

DatasetDict({
    train: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 24
    })
    test: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 24
    })
})

In [8]:
import IPython.display as ipd

audio_sample = data["test"][9]
print(audio_sample["target_text"].lower())
ipd.Audio(data=audio_sample["speech"], autoplay=True, rate=audio_sample["sampling_rate"])

he was touching the dinosaur


In [9]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

processor = Wav2Vec2Processor.from_pretrained(model_fp, cache_dir = cache_fp)
model = Wav2Vec2ForCTC.from_pretrained(model_fp, cache_dir = cache_fp)
#processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h", cache_dir = cache_fp)
#model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h",  cache_dir = cache_fp)
#processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-100h", cache_dir = cache_fp)
#model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-100h", cache_dir = cache_fp)

In [10]:
inputs = processor(audio_sample["speech"], sampling_rate=16_000, return_tensors="pt")

In [11]:
import torch

with torch.no_grad():
    logits = model(**inputs).logits

In [12]:
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)[0]

print(f'Predicted transcription without the language model is:\n{transcription}')

Predicted transcription without the language model is:
HEY WASCOCKING THE DINOSAUR


In [13]:
from transformers import Wav2Vec2ProcessorWithLM

processor1 = Wav2Vec2ProcessorWithLM.from_pretrained("patrickvonplaten/wav2vec2-base-100h-with-lm", cache_dir = cache_fp)

Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 145.54it/s]


In [14]:
logits.shape

torch.Size([1, 133, 32])

In [15]:
" ".join(sorted(processor1.tokenizer.get_vocab()))

"' </s> <pad> <s> <unk> A B C D E F G H I J K L M N O P Q R S T U V W X Y Z |"

In [16]:
transcription = processor1.batch_decode(logits.numpy()).text[0]
print(f'Predicted transcription from the pretrained language model is:\n{transcription}')

Predicted transcription from the pretrained language model is:
HE WAS COCKING THE DINOSAUR


# 2. Getting data for the language model

In [43]:
#full_fp = '/srv/scratch/z5313567/thesis/AusKidTalk_local/AusKidTalk_Storytelling/AusKidTalk_storytelling_texts.csv'
full_fp = '/srv/scratch/z5313567/thesis/AusKidTalk_local/AusKidTalk_Storytelling/AusKiaTalk_CU_storytelling_v2.csv'

In [44]:
from datasets import load_dataset

dataset = load_dataset('csv', 
                        data_files = full_fp,
                        cache_dir = cache_fp)

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 64.58it/s]


Dataset csv downloaded and prepared to /srv/scratch/z5313567/thesis/cache/csv/default-cbbad28bad8cddb0/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 178.90it/s]


In [45]:
dataset

DatasetDict({
    train: Dataset({
        features: ['transcription'],
        num_rows: 34664
    })
})

In [46]:
chars_to_ignore_regex = '[,?.!*\-\;\:\"“%‘”�—’…–]'

In [47]:
import re

def extract_text(batch):
    text = batch["transcription"]
    batch["text"] = re.sub(chars_to_ignore_regex, "", text.lower())
    #batch["text"] = re.sub(chars_to_ignore_regex, "", text.lower())
    return batch

In [48]:
dataset = dataset.map(extract_text, remove_columns=dataset.column_names['train'])
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 34664
    })
})

# 3. Build an n-gram with KenLM

In [49]:
# ################################################################################################################################################################################
# ################################################################################################################################################################################
#
# Do not run the following command, since those dependencies are available as loadable modules on Katana:
# !sudo apt install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev
#
# Instead run the following commands in Putty terminal:
# 1) module load boost/1.80.0 eigen/3.4.0 zlib/1.2.13 bzip2/1.0.6 lzma/4.32.7
# 2) (can run this in JupyterLab notebook, never tried to run this in Putty termianl) ----> !wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz
# 3) !mkdir kenlm/build -p && cd kenlm/build && cmake .. && make -j2
#
# ################################################################################################################################################################################
# ################################################################################################################################################################################

In [50]:
#!wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz

In [51]:
#!mkdir kenlm/build -p && cd kenlm/build && cmake .. && make -j2
#!ls kenlm/build/bin

In [52]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 34664
    })
})

In [53]:
with open("text.txt", "w") as file:
    file.write(" ".join(dataset["train"]["text"]))

In [54]:
# ################################################################################################################################################################################
# ################################################################################################################################################################################
#
# assume 5-gram language model
# run the following commands in Putty terminal
# 1) cd /srv/scratch/z5313567/thesis/JupyterLab/Language_model
# 2) kenlm/build/bin/lmplz -o 5 --verbose header --text text.txt --arpa 5gram.arpa --discount_fallback (Reference: https://blog.csdn.net/w5688414/article/details/106994706)
#    or kenlm/build/bin/lmplz -o 5 <"text.txt" > "5gram.arpa"  
# !! if running 2) results in 'Killed', try running: kenlm/build/bin/lmplz -o 5 -S 50% --discount_fallback --verbose header --text text.txt --arpa 5gram.arpa (Reference: https://blog.csdn.net/wangchengjiea/article/details/102649607)
#
# ################################################################################################################################################################################
# ################################################################################################################################################################################

In [ ]:
#!head -20 5gram.arpa
!head -20 AudKidTalk_4gram_3.arpa


\data\
ngram 1=200003
ngram 2=38229161
ngram 3=45941329
ngram 4=60975692

\1-grams:
-2.348754	</s>
-2.752519	<UNK>	-0.9697837
-99	<s>	-2.408548
-2.619969	A	-1.56262
-7.211563	A''S	-0.1495221
-6.221141	A'BODY	-0.2521624
-6.583487	A'COURT	-0.1844242
-6.240468	A'D	-0.2419162
-7.108924	A'GHA	-0.1740987
-6.260695	A'GOIN	-0.407374
-5.804425	A'LL	-0.3104859
-5.638036	A'M	-0.2983787


In [85]:
# The 5-gram correctly includes a "Unknown" or <unk>, as well as a begin-of-sentence, <s> token, but no end-of-sentence, </s> token. 
# This sadly has to be corrected currently after the build.
# We can simply add the end-of-sentence token by adding the line 0 </s> -0.09522562 below the begin-of-sentence token and increasing the ngram 1 count by 1.

#with open("5gram.arpa", "r") as read_file, open("5gram_correct.arpa", "w") as write_file:
with open("AudKidTalk_4gram_3.arpa", "r") as read_file, open("AudKidTalk_4gram_correct_3.arpa", "w") as write_file:
    has_added_eos = False
    for line in read_file:
        if not has_added_eos and "ngram 1=" in line:
            count=line.strip().split("=")[-1]
            write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
        elif not has_added_eos and "<s>" in line:
            write_file.write(line)
            write_file.write(line.replace("<s>", "</s>"))
            has_added_eos = True
        else:
            write_file.write(line)

In [49]:
#!head -20 5gram_correct.arpa
#!head -20 AudKidTalk_4gram_correct_3.arpa
!head -20 AusKidTalk_LM_combined_v1_lowercase.arpa

\data\
ngram 1=152215
ngram 2=4878850
ngram 3=4396708
ngram 4=1064394

\1-grams:
-99	<s>	-1.31333
-7.51472	'aquila
-6.99997	aasiya	-0.0380279
-6.9031	acidifies
-5.73685	balled	-0.38881
-6.57062	parolin	-0.24223
-7.01366	paroling
-7.13837	paronnaud
-6.23547	paros	-0.111871
-6.48544	parotid	-0.32514
-6.90239	paroubek	-0.192235
-6.07383	paroxetine	-0.41236
-6.28606	paroxysm	-0.224921


# 4. Combine an n-gram with Wav2Vec2

In [1]:
model_fp = '/srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk/progressive_finetune_CU_AusKidTalk_lowercase_freeze_lower_transformer_20231015'

In [2]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained(model_fp)

/home/z5313567/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k: v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

In [4]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path="AusKidTalk_LM_combined_lowercase_v2.arpa"
)

Loading the LM will be faster if you build a binary file.
Reading /srv/scratch/z5313567/thesis/JupyterLab/Language_model/AusKidTalk_LM_combined_lowercase_v2.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
Unigrams and labels don't seem to agree.


In [5]:
(list(sorted_vocab_dict.keys()))

['e',
 'a',
 'h',
 'f',
 'x',
 'm',
 'k',
 'p',
 'o',
 't',
 'l',
 'q',
 'w',
 'r',
 'v',
 'z',
 'c',
 'd',
 'i',
 'g',
 '|',
 's',
 'j',
 'n',
 'b',
 "'",
 'y',
 'u',
 '[UNK]',
 '[PAD]',
 '<s>',
 '</s>']

In [6]:
from transformers import Wav2Vec2ProcessorWithLM

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

In [7]:
processor_with_lm

Wav2Vec2ProcessorWithLM:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "Wav2Vec2Processor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}

- tokenizer: Wav2Vec2CTCTokenizer(name_or_path='/srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk/progressive_finetune_CU_AusKidTalk_lowercase_freeze_lower_transformer_20231015', vocab_size=30, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]', 'additional_special_tokens': [AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), AddedToken("<s>", rstrip=False, lstrip=False, single_word=Fals

In [10]:
lm_save_fp = '/srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk/progressive_finetune_CU_AusKidTalk_lowercase_freeze_lower_transformer_20231015_with_lm_AusKidTalk_LM_combined_lowercase_v2'

In [11]:
import shutil

# Copy the entire source_folder to the destination_folder
shutil.copytree(model_fp, lm_save_fp)

FileExistsError: [Errno 17] File exists: '/srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk/progressive_finetune_CU_AusKidTalk_lowercase_freeze_lower_transformer_20231015_with_lm_AusKidTalk_LM_combined_lowercase_v2'

In [12]:
processor_with_lm.save_pretrained(lm_save_fp)

In [11]:
arpa_fp = /srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk/progressive_finetune_CU_AusKidTalk_lowercase_freeze_lower_transformer_20231015_with_lm_AusKidTalk_LM_combined_lowercase_v2/language_model/AusKidTalk_LM_combined_lowercase_v2.arpa
bin_fp = '/srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk/progressive_finetune_CU_AusKidTalk_lowercase_freeze_base_model_20231004_with_lm_AusKidTalk_LM_combined_lowercase_v2/language_model/AusKidTalk_LM_combined_lowercase_v2.bin'

SyntaxError: invalid decimal literal (2245687693.py, line 1)

In [13]:
#!kenlm/build/bin/build_binary /srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk_scripted_spontaneous_combined/finetune_20230718_with_lm_3/language_model/5gram_correct.arpa /srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk_scripted_spontaneous_combined/finetune_20230718_with_lm_3/language_model/5gram.bin
!kenlm/build/bin/build_binary /srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk/progressive_finetune_CU_AusKidTalk_lowercase_freeze_lower_transformer_20231015_with_lm_AusKidTalk_LM_combined_lowercase_v2/language_model/AusKidTalk_LM_combined_lowercase_v2.arpa /srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk/progressive_finetune_CU_AusKidTalk_lowercase_freeze_lower_transformer_20231015_with_lm_AusKidTalk_LM_combined_lowercase_v2/language_model/AusKidTalk_LM_combined_lowercase_v2.bin

Reading /srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk/progressive_finetune_CU_AusKidTalk_lowercase_freeze_lower_transformer_20231015_with_lm_AusKidTalk_LM_combined_lowercase_v2/language_model/AusKidTalk_LM_combined_lowercase_v2.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [14]:
!rm /srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk/progressive_finetune_CU_AusKidTalk_lowercase_freeze_lower_transformer_20231015_with_lm_AusKidTalk_LM_combined_lowercase_v2/language_model/AusKidTalk_LM_combined_lowercase_v2.arpa